# Métrica 1.1.08: "Congruencia entre estado civil y datos de la pareja"

In [1]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
# Asumimos que tienes un archivo config.py con tus variables de conexión
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_08_ESTADO_CIVIL_VS_PAREJA" 
CATALOG_COLLECTION_NAME = "catalogo_cp"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

# Mapeo de claves de estado civil (Sección 1) a claves de relación (Sección 6)
# Basado en los archivos VarModificación.csv y 1_1_08.csv
ESTADO_CIVIL_A_RELACION = {
    "CAS": "CONYUGE", # Casado(a)
    "CON": "CONCUBINA_CONCUBINARIO_UNION_LIBRE", # Concubinato/Unión Libre
    "SOC": "SOCIEDAD_DE_CONVIVENCIA" # Sociedad de Convivencia
}

# Estados civiles que por definición NO tienen pareja
ESTADOS_SIN_PAREJA = ["SOL", "DIV", "VIU"] # Soltero, Divorciado, Viudo

def validar_estado_civil_vs_pareja(estado_civil_clave: str, datos_pareja_obj: dict) -> str:
    """
    Valida la congruencia entre el estado civil (Sección 1) y los datos
    de la pareja (Sección 6).
    """
    
    # 1. Validamos que tengamos todos los datos de entrada
    if not estado_civil_clave or not isinstance(datos_pareja_obj, dict):
        return "SIN_DATO"

    # 2. Caso: El declarante NO debería tener pareja (Soltero, Divorciado, Viudo)
    if estado_civil_clave in ESTADOS_SIN_PAREJA:
        # Si marcó "ninguno" en la sección de pareja, es coherente.
        # La métrica "CUMPLE" pero no tiene nada que validar, por lo que es "N/A"
        if datos_pareja_obj.get("ninguno") is True:
            return "N/A"
        else:
            # Dijo "Soltero" pero SÍ agregó datos de pareja. Es una incongruencia.
            return "NO_CUMPLE"

    # 3. Caso: El declarante DEBERÍA tener pareja (Casado, Concubinato, etc.)
    if estado_civil_clave in ESTADO_CIVIL_A_RELACION:
        # Si marcó "ninguno" en la sección de pareja, es una incongruencia.
        if datos_pareja_obj.get("ninguno") is True:
            return "NO_CUMPLE"
        
        # Obtenemos la relación que declaró en la Sección 6
        relacion_declarada = datos_pareja_obj.get("relacionConDeclarante", {}).get("clave")
        
        if not relacion_declarada:
            # Dijo "Casado" pero no llenó la Sección 6
            return "SIN_DATO" 
            
        # Comparamos si la relación declarada es la que esperamos
        relacion_esperada = ESTADO_CIVIL_A_RELACION[estado_civil_clave]
        
        if relacion_declarada == relacion_esperada:
            return "CUMPLE"
        else:
            # Dijo "Casado" pero en la Sección 6 puso "Concubinato"
            return "NO_CUMPLE"

    # Si el estado civil no es ninguno de los conocidos (ej. "OTRO")
    return "SIN_DATO"

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_estado_civil():
    """
    Procesa TODAS las declaraciones para la Métrica 1.1.08
    """
    client = None
    total_documentos, cumple, no_cumple, sin_dato, na = 0, 0, 0, 0, 0
    
    try:
        print(f"\nConectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        print(f"--- INICIANDO PROCESAMIENTO (Métrica 1.1.08 - Estado Civil vs Pareja) ---")

        # Proyección optimizada para traer los 2 campos que necesitamos
        proyeccion = {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosGenerales.situacionPersonalEstadoCivil.clave": 1,
            "declaracion.situacionPatrimonial.datosPareja": 1 # Traemos el objeto completo
        }
        
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]
            estado_civil = None
            datos_pareja = None
            
            try:
                # Navegación segura
                datos_generales = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosGenerales", {})
                if datos_generales:
                    estado_civil_obj = datos_generales.get("situacionPersonalEstadoCivil")
                    if isinstance(estado_civil_obj, dict):
                        estado_civil = estado_civil_obj.get("clave")

                datos_pareja = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosPareja")

            except AttributeError:
                pass 

            resultado_metrica = validar_estado_civil_vs_pareja(estado_civil, datos_pareja)
            
            # Actualizamos contadores
            if resultado_metrica == "CUMPLE": cumple += 1
            elif resultado_metrica == "NO_CUMPLE": no_cumple += 1
            elif resultado_metrica == "N/A": na += 1
            else: sin_dato += 1 # SIN_DATO o cualquier otro

            # --- CONSERVAMOS TU SALIDA DE CONSOLA ---
            print(f"\nProcesando Documento ID: {original_id}")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > Estado Civil (Clave): '{estado_civil}'")
            print(f"  > Datos Pareja ('ninguno'): {datos_pareja.get('ninguno') if isinstance(datos_pareja, dict) else 'N/D'}")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")

            filtro = { "_id": original_id }
            actualizacion = { "$set": { METRIC_ID: resultado_metrica } }
            target_collection.update_one(filtro, actualizacion, upsert=True)
            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.1.08):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {cumple}")
        print(f"  > Métrica 'NO_CUMPLE': {no_cumple}")
        print(f"  > Métrica 'N/A' (Sin pareja): {na}")
        print(f"  > Métrica 'SIN_DATO': {sin_dato}")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e: print(f"Ocurrió un error inesperado: {e}")
    finally:
        if client: client.close(); print("Conexión cerrada.")

if __name__ == "__main__":
    procesar_todas_las_declaraciones_estado_civil()


Conectando a MongoDB en sistema1...
¡Conexión exitosa!
--- INICIANDO PROCESAMIENTO (Métrica 1.1.08 - Estado Civil vs Pareja) ---

Procesando Documento ID: 68f81b8800535f910a29f694
--- RESULTADO DEL ANÁLISIS ---
  > Estado Civil (Clave): 'None'
  > Datos Pareja ('ninguno'): N/D
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695
--- RESULTADO DEL ANÁLISIS ---
  > Estado Civil (Clave): 'None'
  > Datos Pareja ('ninguno'): N/D
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696
--- RESULTADO DEL ANÁLISIS ---
  > Estado Civil (Clave): 'None'
  > Datos Pareja ('ninguno'): N/D
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'met

KeyboardInterrupt: 